In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, UpSampling2D, Add, LeakyReLU
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import zipfile
from  rgb_lab_formulation import *

In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/Colourize/wider Face Datset/WIDER_train.zip") as zfile:
    zfile.extractall()

datasets1 = tf.keras.preprocessing.image_dataset_from_directory("/content/WIDER_train/", batch_size=8, image_size=(256,256))

Found 12880 files belonging to 1 classes.


In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/Colourize/wider Face Datset/WIDER_test.zip") as zfile:
    zfile.extractall()

datasets2 = tf.keras.preprocessing.image_dataset_from_directory("/content/WIDER_test/", batch_size=8, image_size=(256,256))

Found 16097 files belonging to 1 classes.


In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/Colourize/wider Face Datset/WIDER_val.zip") as zfile:
    zfile.extractall()
    
datasets3 = tf.keras.preprocessing.image_dataset_from_directory("/content/WIDER_val/", batch_size=8, image_size=(256,256))

Found 3226 files belonging to 1 classes.


In [ ]:
ds = datasets1.concatenate(datasets2)
ds = ds.concatenate(datasets3).unbatch()
norm_ds = ds.map(lambda x,y : x/255)

In [ ]:
processed_ds = norm_ds.map(lambda x : (tf.concat((rgb_to_lab(x)[:,:,:1]/100, rgb_to_lab(x)[:,:,:1]/100,rgb_to_lab(x)[:,:,:1]/100), axis=-1), 
                             rgb_to_lab(x)[:,:,1:]/128))
processed_ds = processed_ds.shuffle(1000)

In [ ]:
train_ds = processed_ds.skip(500).batch(8)
test_ds = processed_ds.take(500).batch(8)

In [ ]:
def resuidal_block(vgg_layer_out, prev_layer, upscale_size=2):
    up_scaled = UpSampling2D(size=upscale_size)(prev_layer)
    vgg_layer_out = BatchNormalization(axis=-1)(vgg_layer_out)
    resuidal_add = Add()([up_scaled, vgg_layer_out])
    return resuidal_add

def ColouriseModel(input_shape=[256,256,3]):
    vgg = vgg19.VGG19(include_top=False, input_shape=input_shape)
    vgg.trainable = False

    outputs = {}
    outputs['0_inputs'] = vgg.layers[0].output
    concat_layers_name = ['block1_conv2', 'block2_conv2', 'block3_conv4', 'block4_conv4', 'block5_conv4']
    for name in concat_layers_name:
        outputs[name] = vgg.get_layer(name).output


    vgg_last_out_norm = BatchNormalization(axis=-1)(outputs['block5_conv4'])
    conv = Conv2D(512, (1,1), padding='SAME')(vgg_last_out_norm)

    resuidal_add_out = resuidal_block(outputs['block4_conv4'], conv)
    resuidal_out = Conv2D(256, (1,1), padding='SAME')(resuidal_add_out)
    activation_lrelu = LeakyReLU()(resuidal_out)

    resuidal_add_out = resuidal_block(outputs['block3_conv4'], activation_lrelu)
    resuidal_out = Conv2D(128, (1,1), padding='SAME')(resuidal_add_out)
    activation_lrelu = LeakyReLU()(resuidal_out)

    resuidal_add_out = resuidal_block(outputs['block2_conv2'], activation_lrelu)
    resuidal_out = Conv2D(64, (1,1), padding='SAME')(resuidal_add_out)
    activation_lrelu = LeakyReLU()(resuidal_out)

    resuidal_add_out = resuidal_block(outputs['block1_conv2'], activation_lrelu)
    resuidal_out = Conv2D(3, (1,1), padding='SAME')(resuidal_add_out)
    activation_lrelu = LeakyReLU()(resuidal_out)

    resuidal_add_out = resuidal_block(outputs['0_inputs'], activation_lrelu, upscale_size=1)
    resuidal_out = Conv2D(3, (1,1), padding='SAME')(resuidal_add_out)
    activation_lrelu = LeakyReLU()(resuidal_out)

    model_out = Conv2D(2, (1,1), activation='tanh', padding='SAME')(activation_lrelu)


    return tf.keras.Model([vgg.input], model_out)

In [ ]:
model = ColouriseModel()

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae', "mse"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [ ]:
model.fit(train_ds, epochs=3, validation_data=test_ds)

Epoch 1/3
3963/3963 [==============================] - 1643s 398ms/step - loss: 0.0218 - mae: 0.0946 - mse: 0.0218 - val_loss: 0.0134 - val_mae: 0.0763 - val_mse: 0.0134
Epoch 2/3
3963/3963 [==============================] - 1563s 382ms/step - loss: 0.0130 - mae: 0.0751 - mse: 0.0130 - val_loss: 0.0120 - val_mae: 0.0722 - val_mse: 0.0120
Epoch 3/3
3963/3963 [==============================] - 1568s 383ms/step - loss: 0.0129 - mae: 0.0745 - mse: 0.0129 - val_loss: 0.0127 - val_mae: 0.0761 - val_mse: 0.0127


In [ ]:
model.save("/content/drive/MyDrive/Colourize/v2model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colourize/v2model/assets
